In [1]:
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

In [2]:
import plotly
plotly.__version__

'2.0.16'

In [3]:
# Copyright (c) 2011, Mark Chenoweth
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification, are permitted 
# provided that the following conditions are met:
#
# - Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
#
# - Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following 
#   disclaimer in the documentation and/or other materials provided with the distribution.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, 
# INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE 
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL,
# EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS 
# OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, 
# STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF 
# ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [5]:
import urllib,time,datetime
import  pandas as pd


class Quote(object):
    DATE_FMT = '%Y-%m-%d'
    TIME_FMT = '%H:%M:%S'
  
    def __init__(self):
        self.symbol = ''
        self.date,self.time,self.open_,self.high,self.low,self.close,self.volume = ([] for _ in range(7))
    
    def append(self,dt,open_,high,low,close,volume):
        self.date.append(dt.date())
        self.time.append(dt.time())
        self.open_.append(float(open_))
        self.high.append(float(high))
        self.low.append(float(low))
        self.close.append(float(close))
        self.volume.append(int(volume)) 
    
    def to_csv(self):
        return ''.join(["{0},{1},{2},{3:.2f},{4:.2f},{5:.2f},{6:.2f},{7}\n".format(self.symbol,
              self.date[bar].strftime('%Y-%m-%d'),self.time[bar].strftime('%H:%M:%S'),
              self.open_[bar],self.high[bar],self.low[bar],self.close[bar],self.volume[bar]) 
              for bar in xrange(len(self.close))])
    
    def write_csv(self,filename):
        with open(filename,'w') as f:
              f.write(self.to_csv())   
               
    def read_csv(self,filename):
        self.symbol = ''
        self.date,self.time,self.open_,self.high,self.low,self.close,self.volume = ([] for _ in range(7))
        for line in open(filename,'r'):
            symbol,ds,ts,open_,high,low,close,volume = line.rstrip().split(',')
            self.symbol = symbol
            dt = datetime.datetime.strptime(ds+' '+ts,self.DATE_FMT+' '+self.TIME_FMT)
            self.append(dt,open_,high,low,close,volume)
        return True  
    
    def __repr__(self):
        return self.to_csv()

In [16]:
class GoogleIntradayQuote(Quote):
    ''' Intraday quotes from Google. Specify interval seconds and number of days '''
    def __init__(self,symbol,interval_seconds=300,num_days=5):
        super(GoogleIntradayQuote,self).__init__()
        self.symbol = symbol.upper()
        url_string = "http://www.google.com/finance/getprices?q={0}".format(self.symbol)
        url_string += "&x=NSE&i={0}&p={1}d&f=d,o,h,l,c,v".format(interval_seconds,num_days)
        csv = urllib.request.urlopen(url_string).readlines()
        for bar in xrange(7,len(csv)):
            if csv[bar].count(',')!=5: continue
            offset,close,high,low,open_,volume = csv[bar].split(',')
            if offset[0]=='a':
                day = float(offset[1:])
                offset = 0
            else:
                offset = float(offset)
            open_,high,low,close = [float(x) for x in [open_,high,low,close]]
            dt = datetime.datetime.fromtimestamp(day+(interval_seconds*offset))
            self.append(dt,open_,high,low,close,volume)

if __name__ == '__main__':
    q = GoogleIntradayQuote('RCOM',300,30)
    # print it out
    q.to_csv('rcom.csv') 

HTTPError: HTTP Error 403: Forbidden

In [13]:
import os
print(os.getcwd())
os.chdir('C:\\Users\\G560667\\stock-trader\\')
print(os.getcwd())

\\inmbfp1\redirect1\G560667\My Documents
C:\Users\G560667\stock-trader


In [14]:
import pandas as pd
from pandas import read_csv
from matplotlib import pyplot as plt

In [21]:
import json

import requests

rsp = requests.get('https://finance.google.com/finance?q=AAPL&output=json')

if rsp.status_code in (200,):

    # This magic here is to cut out various leading characters from the JSON 
    # response, as well as trailing stuff (a terminating ']\n' sequence), and then
    # we decode the escape sequences in the response
    # This then allows you to load the resulting string
    # with the JSON module.
    fin_data = json.loads(rsp.content[6:-2].decode('unicode_escape'))

    # print out some quote data
    print('Opening Price: {}'.format(fin_data['op']))
    print('Price/Earnings Ratio: {}'.format(fin_data['pe']))
    print('high: {}'.format(fin_data['hi']))
    print('low: {}'.format(fin_data['lo']))

Opening Price: 174.35
Price/Earnings Ratio: 18.97
high: 175.61
low: 174.14


In [33]:


import csv
import datetime
import re

import pandas as pd
import requests

def get_google_finance_intraday(ticker, period=60, days=1):
    """
    Retrieve intraday stock data from Google Finance.
    Parameters
    ----------
    ticker : str
        Company ticker symbol.
    period : int
        Interval between stock values in seconds.
    days : int
        Number of days of data to retrieve.
    Returns
    -------
    df : pandas.DataFrame
        DataFrame containing the opening price, high price, low price,
        closing price, and volume. The index contains the times associated with
        the retrieved price values.
    """

    uri = 'http://www.google.com/finance/getprices' \
          '?x=NASDAQ?i={period}&p={days}d&f=d,o,h,l,c,v&df=cpct&q={ticker}'.format(ticker=ticker,
                                                                          period=period,
                                                                          days=days)
    page = requests.get(uri)
    reader = csv.reader(page.content.splitlines())
    columns = ['Close', 'High', 'Low', 'Open', 'Volume']
    rows = []
    times = []
    print(reader)
    reader = reader.decode('utf8')
    for row in reader:
        if re.match('^[a\d]', row[0]):
            if row[0].startswith('a'):
                start = datetime.datetime.fromtimestamp(int(row[0][1:]))
                times.append(start)
            else:
                times.append(start+datetime.timedelta(seconds=period*int(row[0])))
            rows.append(map(float, row[1:]))
    if len(rows):
        return pd.DataFrame(rows, index=pd.DatetimeIndex(times, name='Date'),
                            columns=columns)
    else:
        return pd.DataFrame(rows, index=pd.DatetimeIndex(times, name='Date'))

In [34]:
get_google_finance_intraday('AAPL',60,5)

AttributeError: '_csv.reader' object has no attribute 'decode'